<a href="https://colab.research.google.com/github/MarioBolanos/AnalisisNumerico/blob/master/5_MachineLearning/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tomado de mplement a neural network from scratch with Python/Numpy — Backpropagation
# Ali Mirzaei / Jan 19, 2019
# https://medium.com/@a.mirzaei69/implement-a-neural-network-from-scratch-with-python-numpy-backpropagation-e82b70caa9bb

import numpy as np
class NeuralNetwork(object):
    def __init__(self, layers = [2 , 10, 1], activations=['sigmoid', 'sigmoid']):
        assert(len(layers) == len(activations)+1)
        self.layers = layers
        self.activations = activations
        self.weights = []
        self.biases = []
        for i in range(len(layers)-1):
            self.weights.append(np.random.randn(layers[i+1], layers[i]))
            self.biases.append(np.random.randn(layers[i+1], 1))
    
    def feedforward(self, x):
        # return the feedforward value for x
        a = np.copy(x)
        z_s = []
        a_s = [a]
        for i in range(len(self.weights)):
            activation_function = self.getActivationFunction(self.activations[i])
            z_s.append(self.weights[i].dot(a) + self.biases[i])
            a = activation_function(z_s[-1])
            a_s.append(a)
        return (z_s, a_s)

    def backpropagation(self,y, z_s, a_s):
        dw = []  # dC/dW
        db = []  # dC/dB
        deltas = [None] * len(self.weights)  # delta = dC/dZ  known as error for each layer
        # insert the last layer error
        deltas[-1] = ((y-a_s[-1])*(self.getDerivitiveActivationFunction(self.activations[-1]))(z_s[-1]))
        # Perform BackPropagation
        for i in reversed(range(len(deltas)-1)):
            deltas[i] = self.weights[i+1].T.dot(deltas[i+1])*(self.getDerivitiveActivationFunction(self.activations[i])(z_s[i]))        
        #a= [print(d.shape) for d in deltas]
        batch_size = y.shape[1]
        db = [d.dot(np.ones((batch_size,1)))/float(batch_size) for d in deltas]
        dw = [d.dot(a_s[i].T)/float(batch_size) for i,d in enumerate(deltas)]
        # return the derivitives respect to weight matrix and biases
        return dw, db

    def train(self, x, y, batch_size=10, epochs=100, lr = 0.01):
        # update weights and biases based on the output
        e_print=0
        for e in range(epochs): 
            i=0
            while(i<len(y)):
                x_batch = x[i:i+batch_size]
                y_batch = y[i:i+batch_size]
                i = i+batch_size
                z_s, a_s = self.feedforward(x_batch)
                dw, db = self.backpropagation(y_batch, z_s, a_s)
                self.weights = [w+lr*dweight for w,dweight in  zip(self.weights, dw)]
                self.biases = [w+lr*dbias for w,dbias in  zip(self.biases, db)]
                if e >= e_print:
                  e_print = e_print+int(epochs/10)
                  print("epoch = "+str(e)+", "+"loss = {}".format(np.linalg.norm(a_s[-1]-y_batch) ))
    @staticmethod
    def getActivationFunction(name):
        if(name == 'sigmoid'):
            return lambda x : np.exp(x)/(1+np.exp(x))
        elif(name == 'linear'):
            return lambda x : x
        elif(name == 'relu'):
            def relu(x):
                y = np.copy(x)
                y[y<0] = 0
                return y
            return relu
        else:
            print('Unknown activation function. linear is used')
            return lambda x: x
    
    @staticmethod
    def getDerivitiveActivationFunction(name):
        if(name == 'sigmoid'):
            sig = lambda x : np.exp(x)/(1+np.exp(x))
            return lambda x :sig(x)*(1-sig(x)) 
        elif(name == 'linear'):
            return lambda x: 1
        elif(name == 'relu'):
            def relu_diff(x):
                y = np.copy(x)
                y[y>=0] = 1
                y[y<0] = 0
                return y
            return relu_diff
        else:
            print('Unknown activation function. linear is used')
            return lambda x: 1
 

In [ ]:
import numpy as np


In [ ]:
my_nn = NeuralNetwork([2,1,1],activations=['linear', 'linear'])

In [ ]:
# my_nn.weights = np.array([[0.5, -0.5], [0]])
# my_nn.biases = np.array([[ 1], [0]])
# my_nn.activations = ['linear','linear']
print(my_nn.weights)
print(my_nn.biases)
print(my_nn.activations)

In [ ]:
X = np.array([[0., 0.],[0., 1.],[1.,0.],[1.,1.]])
# print(X.transpose())
z_s, a_s = my_nn.feedforward(X.transpose())

# print(z_s)
# print(a_s)
print(a_s[0])
print(a_s[2])

In [ ]:
import timeit

y = np.array([[-1,1,1,1]])

start = timeit.timeit()

my_nn.train(X.transpose(), y, epochs=1000, batch_size=4, lr = .1)

end = timeit.timeit()
print("Elapsed time "+str(end - start))

z_s, a_s = my_nn.feedforward(X.transpose())

# print(z_s)
# print(a_s)
print(a_s[0])
print(a_s[2])
print(my_nn.weights)
print(my_nn.biases)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect(1)
i=0
for dato in X:
  c = 'r' if y[0][i] > 0 else 'b'
  i+=1
  ax.scatter(dato[0],dato[1], color=c)



# print(z_s)
# print(a_s)
# print(a_s[0])
# print(a_s[2])
# xs = np.array([-1,2])
# ys = np.array([2,-1])
# ys1 = -(my_nn.weights[0][0][0]*xs+my_nn.biases[0][0][0])/my_nn.weights[0][0][0]
# print(ys1)
# ys= my_nn.weights[1][0][0]*ys1+my_nn.biases[1][0][0]
# print(ys)
# ax.plot(xs,ys,color='k')


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
X_test = np.random.rand(400).reshape(200, 2)

z_s, a_s = my_nn.feedforward(X_test.transpose())
y_test = a_s[2]
# print(X_test.transpose())
# print(y_test)
i=0
for dato in X_test:
  c = 'r' if y_test[0][i] > 0.5 else 'b'
  i+=1
  ax.scatter(dato[0],dato[1], color=c)

In [ ]:
# if __name__=='__main__':
import matplotlib.pyplot as plt
nn = NeuralNetwork([1, 10, 1],activations=['sigmoid', 'linear'])
X = 2*np.pi*np.random.rand(5).reshape(1, -1)
y = np.sin(X)
print(X.size,X)
print(y.size,y)
z_s, a_s = nn.feedforward(X)
# print(z_s)
# print(a_s)
# print(a_s[0])
print(a_s[2])

In [ ]:
nn.train(X, y, epochs=1000, batch_size=64, lr = .1)
z_s, a_s = nn.feedforward(X)
#print(y, X)
plt.scatter(X.flatten(), y.flatten())
plt.scatter(X.flatten(), a_s[-1].flatten())
plt.show()
# print(z_s)
# print(a_s)
print(y)
print(a_s[2])

In [ ]:
nn.train(X, y, epochs=1000, batch_size=64, lr = .1)
z_s, a_s = nn.feedforward(X)
#print(y, X)
plt.scatter(X.flatten(), y.flatten())
plt.scatter(X.flatten(), a_s[-1].flatten())
plt.show()

In [ ]:
### imports
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
### constant data
x  = [[0.,0.],[1.,1.],[1.,0.],[0.,1.]]
y_ = [[0.],[0.],[1.],[1.]]

### induction
# 1x2 input -> 2x3 hidden sigmoid -> 3x1 sigmoid output

# Layer 0 = the x2 inputs
x0 = tf.constant( x  , dtype=tf.float32 )
y0 = tf.constant( y_ , dtype=tf.float32 )

# Layer 1 = the 2x3 hidden sigmoid
m1 = tf.Variable( tf.random_uniform( [2,3] , minval=0.1 , maxval=0.9 , dtype=tf.float32  ))
b1 = tf.Variable( tf.random_uniform( [3]   , minval=0.1 , maxval=0.9 , dtype=tf.float32  ))
h1 = tf.sigmoid( tf.matmul( x0,m1 ) + b1 )

# Layer 2 = the 3x1 sigmoid output
m2 = tf.Variable( tf.random_uniform( [3,1] , minval=0.1 , maxval=0.9 , dtype=tf.float32  ))
b2 = tf.Variable( tf.random_uniform( [1]   , minval=0.1 , maxval=0.9 , dtype=tf.float32  ))
y_out = tf.sigmoid( tf.matmul( h1,m2 ) + b2 )


### loss
# loss : sum of the squares of y0 - y_out
loss = tf.reduce_sum( tf.square( y0 - y_out ) )

# training step : gradient decent (1.0) to minimize loss
train = tf.train.GradientDescentOptimizer(1.0).minimize(loss)


### training
# run 500 times using all the X and Y
# print out the loss and any other interesting info
with tf.Session() as sess:
  sess.run( tf.global_variables_initializer() )
  for step in range(500) :
    sess.run(train)

  results = sess.run([m1,b1,m2,b2,y_out,loss])
  labels  = "m1,b1,m2,b2,y_out,loss".split(",")
  for label,result in zip(*(labels,results)) :
    print ("")
    print (label)
    print (result)

print ("")

In [ ]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Librerias de ayuda
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)